#### Layer Normalization部分

In [ ]:
def ln(inputs, epsilon = 1e-8, scope="ln"):
    """
    使用层归一layer normalization
    tensorflow 在实现 Batch Normalization（各个网络层输出的归一化）时，主要用到nn.moments和batch_normalization
    其中moments作用是统计矩，mean 是一阶矩，variance 则是二阶中心矩
    tf.nn.moments 计算返回的 mean 和 variance 作为 tf.nn.batch_normalization 参数进一步调用
    :param inputs: 一个有2个或更多维度的张量，第一个维度是batch_size
    :param epsilon: 很小的数值，防止区域划分错误
    :param scope:
    :return: 返回一个与inputs相同shape和数据的dtype
    """
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]

        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.get_variable("beta", params_shape, initializer=tf.zeros_initializer())
        gamma = tf.get_variable("gamma", params_shape, initializer=tf.ones_initializer())
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta

    return outputs

#### Mask部分
 Padding Mask 与 Sequence Mask

`tf.reduce_sum()`: 计算张量tensor沿着某一维度的和，可以在求和后降维

`tf.sign()`: 返回符号(-1, 0, 1), y= sign(x) = -1 if x<0; 0 if x==0; 1 if x>0.

`tf.expand_dims()`: 使用axis参数。给定一个张量输入，这个操作在输入形状的维数索引轴上插入一个维数为1的维度

`tf.tile()`: 平铺之意，用于在同一维度上的复制，multiples参数维度与input维度应一致

`tf.where(input, a, b)`: a，b均为尺寸一致的tensor，实现a中对应input中true的位置的元素值不变，其余元素由b中对应位置元素替换
 

In [ ]:
def mask(inputs, queries=None, keys=None, type=None):
    """
    对Keys或Queries进行遮盖
    :param inputs: (N, T_q, T_k)
    :param queries: (N, T_q, d)
    :param keys: (N, T_k, d)
    :return:

    e.g.,
    >> queries = tf.constant([[[1.],
                               [2.],
                               [0.]]], tf.float32) # (1, 3, 1)
    >> keys = tf.constant([[[4.],
                            [0.]]], tf.float32)  # (1, 2, 1)
    >> inputs = tf.constant([[[4., 0.],
                              [8., 0.],
                              [0., 0.]]], tf.float32)
    >> mask(inputs, queries, keys, "key")
    array([[[ 4.0000000e+00, -4.2949673e+09],
            [ 8.0000000e+00, -4.2949673e+09],
            [ 0.0000000e+00, -4.2949673e+09]]], dtype=float32)
    
    >> inputs = tf.constant([[[1., 0.],
                              [1., 0.],
                              [1., 0.]]], tf.float32)
    >> mask(inputs, queries, keys, "query")
    array([[[1., 0.],
            [1., 0.],
            [0., 0.]]], dtype=float32)
    """

    padding_num = -2 ** 32 + 1
    if type in ("k", "key", "keys"):
        # Generate masks
        masks = tf.sign(tf.reduce_sum(tf.abs(keys), axis=-1))  # (N, T_k)
        masks = tf.expand_dims(masks, 1) # (N, 1, T_k)
        masks = tf.tile(masks, [1, tf.shape(queries)[1], 1])  # (N, T_q, T_k)

        # Apply masks to inputs
        paddings = tf.ones_like(inputs) * padding_num
        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)  # (N, T_q, T_k)
    elif type in ("q", "query", "queries"):
        # Generate masks
        masks = tf.sign(tf.reduce_sum(tf.abs(queries), axis=-1))  # (N, T_q)
        masks = tf.expand_dims(masks, -1)  # (N, T_q, 1)
        masks = tf.tile(masks, [1, 1, tf.shape(keys)[1]])  # (N, T_q, T_k)

        # Apply masks to inputs
        outputs = inputs*masks
    elif type in ("f", "future", "right"):
        diag_vals = tf.ones_like(inputs[0, :, :])  # (T_q, T_k)
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()  # (T_q, T_k)
        masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(inputs)[0], 1, 1])  # (N, T_q, T_k)

        paddings = tf.ones_like(masks) * padding_num
        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)
    else:
        print("Check if you entered type correctly!")


    return outputs